In [ ]:
import json
import requests
import pandas as pd
import numpy as np
import sqlite3
import seaborn as sns
from matplotlib import pyplot as plt
import uuid
import re
from typing import Callable

In [184]:
API_BASE_URL = "http://localhost:8000"

ROLE_MAP = {
    12: 'start',
    13: 'hand',
    14: 'finish',
    15: 'foot'
}

def df_sql(db_name, query,index_col='uuid'):
    """Extract table <table_name> from database <db_name>.db"""
    with sqlite3.connect(f"data/boardlib/{db_name}.db") as conn:
        df = pd.read_sql_query(query,conn,index_col=index_col)
    return df

def convert_dataframe_to_holds(df: pd.DataFrame, 
                                 default_pull_x: float = 0.0,
                                 default_pull_y: float = -1.0,
                                 default_useability: float = 0.5) -> list[dict]:
    """
    Convert pandas DataFrame to list of hold dictionaries.
    
    Args:
        df: DataFrame with 'hole_id', 'x_ft', 'y_ft' columns
        default_pull_x: Default pull direction x-component
        default_pull_y: Default pull direction y-component  
        default_useability: Default difficulty rating (0-1)
        default_is_foot: Whether hold is a foothold (0 or 1)
    
    Returns:
        List of hold dictionaries matching the API schema
    """
    holds = []
    
    for _, row in df.iterrows():
        hold = {
            "hold_index": int(row['hole_id']),
            "x": float(row['x_ft']),
            "y": float(row['y_ft']),
            "pull_x": default_pull_x,
            "pull_y": default_pull_y,
            "useability": default_useability,
            "is_foot": int(row['is_foot'])
        }
        holds.append(hold)
    
    return holds


def upload_holds(wall_id: str, holds: list[dict], api_base_url: str = API_BASE_URL) -> dict:
    """
    Upload holds to the API.
    
    Args:
        wall_id: The wall ID to upload holds to
        holds: List of hold dictionaries
        api_base_url: Base URL of the API
    
    Returns:
        Response JSON from the API
    """
    endpoint = f"{api_base_url}/api/v1/walls/{wall_id}/holds"
    
    # Prepare form data with JSON-encoded holds
    form_data = {
        "holds": json.dumps(holds)
    }
    
    print(f"Uploading {len(holds)} holds to wall {wall_id}...")
    print(f"Endpoint: {endpoint}")
    
    response = requests.put(endpoint, data=form_data)
    
    if response.status_code == 201:
        print(f"✓ Successfully uploaded {len(holds)} holds!")
        return response.json()
    else:
        print(f"✗ Upload failed with status {response.status_code}")
        print(f"Error: {response.text}")
        response.raise_for_status()

def parse_frames_to_holdset(frames: str, p_h_dict: dict) -> dict:
    """
    Parse frames string to holdset dict.
    
    Format: "p{hold_idx}r{role}p{hold_idx}r{role}..."
    Role mapping: 5=start, 6=hand, 7=finish, 8=foot
    """

    holdset = {
        'start': [],
        'finish': [],
        'hand': [],
        'foot': []
    }
    
    # Find all p{int}r{int} patterns
    pattern = r'p(\d+)r(\d+)'
    matches = re.findall(pattern, frames)
    
    for p_str, role_str in matches:
        p = int(p_str)
        hole = p_h_dict[p]
        role = int(role_str)
        if role in ROLE_MAP:
            holdset[ROLE_MAP[role]].append(hole)
    
    return holdset


def upload_climbs_batch(
    df: pd.DataFrame,
    wall_id: str,
    hole_dict: dict,
    base_url: str = API_BASE_URL,
    batch_size: int = 500,
    verbose: bool = True,
    try_one: bool = False
) -> list[dict]:
    """
    Upload climbs from DataFrame to backend using batch endpoint.
    
    Args:
        df: DataFrame with columns [uuid, angle, frames, difficulty_average, 
            quality_average, ascensionist_count, fa_username, created_at]
        wall_id: The wall ID to upload climbs to
        base_url: Base URL of the API
        batch_size: Number of climbs per batch request
        verbose: Print progress updates
        try_one: If True, only process the first row (for testing)
    
    Returns:
        List of results with original uuid and response/error
    """
    endpoint = f"{base_url}/api/v1/walls/{wall_id}/climbs/batch"
    results = []
    
    if try_one:
        df = df.head(1)
    
    total = len(df)
    
    # Process in batches
    for batch_start in range(0, total, batch_size):
        batch_end = min(batch_start + batch_size, total)
        batch_df = df.iloc[batch_start:batch_end]
        
        # Build payloads for this batch, tracking original uuids
        climbs = []
        batch_uuids = []
        
        for idx, row in batch_df.iterrows():
            try:
                # Parse frames to holdset
                holdset = parse_frames_to_holdset(row['frames'],hole_dict)
                
                # Build payload (same logic as original)
                payload = {
                    'name': f"{row['name']}-{row['angle']}",
                    'holdset': holdset,
                    'angle': int(row['angle']),
                    'grade': float(row['difficulty_average']),
                    'quality': float(row['quality_average']),
                    'ascents': int(row['ascensionist_count']),
                    'setter_name': row.get('fa_username') if pd.notna(row.get('fa_username')) else None,
                    'tags': None
                }
                
                climbs.append(payload)
                batch_uuids.append(row['name'])
                
            except Exception as e:
                # If payload construction fails, record error immediately
                results.append({
                    'original_uuid': row['name'],
                    'status': 'error',
                    'error': f"Payload construction failed: {str(e)}"
                })
        
        # Send batch request if we have climbs
        if climbs:
            try:
                response = requests.post(endpoint, json={'climbs': climbs})
                
                if response.status_code == 201:
                    batch_results = response.json()['results']
                    
                    # Map batch results back to per-row format
                    for result in batch_results:
                        idx = result['index']
                        if result['status'] == 'success':
                            results.append({
                                'original_uuid': batch_uuids[idx],
                                'new_id': result['id'],
                                'status': 'success'
                            })
                        else:
                            results.append({
                                'original_uuid': batch_uuids[idx],
                                'status': 'error',
                                'error': result.get('error', 'Unknown error')
                            })
                else:
                    # Entire batch request failed
                    for uuid in batch_uuids:
                        results.append({
                            'original_uuid': uuid,
                            'status': 'error',
                            'error': f"Batch request failed: {response.text}"
                        })
                        
            except Exception as e:
                # Network/request error - mark all in batch as failed
                for uuid in batch_uuids:
                    results.append({
                        'original_uuid': uuid,
                        'status': 'error',
                        'error': f"Request failed: {str(e)}"
                    })
        
        # Progress update
        if verbose:
            success_count = sum(1 for r in results if r['status'] == 'success')
            print(f"Progress: {len(results)}/{total} | Success: {success_count} | Errors: {len(results) - success_count}")
    
    return results

In [ ]:
query='SELECT * FROM holes'
df_holes = df_sql("kilter",query, index_col='id').astype({'product_id':int, 'name':str,'x':int,'y':int,'mirrored_hole_id':int,'mirror_group':int})
df_holes.head()

In [ ]:
fig, axes = plt.subplots(3,3,figsize=(8,8), sharex=True, sharey=True)

for i,product_id in enumerate(set(df_holes['product_id'])):
    ax = axes[i//3][i%3]
    filtered = df_holes[df_holes["product_id"]==product_id]
    sns.scatterplot(filtered, x='x',y='y',ax=ax)
    ax.set_title(f"Product ID: {product_id}")
    ax.grid(True,alpha=.6, linestyle='--')

plt.show()

### Product 4 is the TB1, Product 5 is the TB2. We need to convert both of these hold-sets to real, physical holds. First step would be aligning them with their real, physical location on the board. Second step would be finding some way to manually add pull_dir+useability features.

I learned from the TB2 installation guide that these hole location measurements are in inches. Additionally, the TB2 DOESN'T HAVE A KICKBOARD! That's crazy, and a bit of a Mandela effect for me. I could swear it had one. But nope.

**The guide also comes with crystal clear hold orientations, and difficulty levels. This is exactly the data I'm looking for! Fuck yeah, Tension!** 

In [ ]:
query = "SELECT * FROM placements WHERE layout_id = 1"
df_placements = df_sql("kilter",query,index_col=None).astype({'id': int, 'layout_id':int,'hole_id':int,'set_id':int,'default_placement_role_id':int})
df_placements = df_placements.merge(
    df_holes,
    left_on='hole_id',
    right_on='id',
    how='inner'
)
# y_too_high = df_placements[df_placements['y'] > 152]
# df_placements.loc[df_placements['y'] > 152,['x','y']] = np.array([[int(n) for n in l]for l in y_too_high['name'].str.split(',')])
# kb = df_placements[df_placements['name'].apply(lambda x: any(kb in x.split(',') for kb in ['KB1','KB2']))]
# df_placements.loc[df_placements['name'].apply(lambda x: any(kb in x.split(',') for kb in ['KB1','KB2'])),'x'] = np.array([int(l[0]) for l in kb['name'].str.split(',')])
df_placements = df_placements[(df_placements['y'] <= 152)]
df_placements['x_ft'] = (df_placements['x']+24)/12
df_placements['y_ft'] = (df_placements['y'])/12
df_placements['is_foot'] = (df_placements['default_placement_role_id']%4==3).astype(int)
# df_placements[(df_placements['y_ft'] < 4) & (df_placements['y_ft']> 3) & (df_placements['x_ft'] < 4) & (df_placements['x_ft'] > 3)].sort_values('x')
df_placements

In [ ]:
kilter_holds = convert_dataframe_to_holds(df_placements)
upload_holds('wall-e77a688049fb',kilter_holds)

In [ ]:
query="""
        SELECT 
            c.uuid,
            c.name,
            c.layout_id,
            cs.angle,
            c.frames,
            cs.difficulty_average,
            cs.quality_average,
            cs.ascensionist_count,
            cs.fa_username,
            c.created_at
        FROM climbs c
        JOIN climb_stats cs ON c.uuid = cs.climb_uuid
        WHERE c.layout_id = 1
            AND cs.angle IS NOT NULL
            AND cs.ascensionist_count > 0
            AND cs.difficulty_average IS NOT NULL
            AND cs.quality_average IS NOT NULL
        ORDER BY cs.ascensionist_count DESC;
    """
df_climbs = df_sql("kilter",query)
df_climbs.head()

In [ ]:
ph_dict = dict(zip(df_placements.loc[df_placements['layout_id']==1,'id'],df_placements['hole_id']))


In [ ]:
upload_climbs_batch(df_climbs, wall_id='wall-e77a688049fb',hole_dict=ph_dict)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
grades_dict = {}
for i, row in df_climbs[df_climbs['ascensionist_count'] > 1].iterrows():
    try:
        holdset = parse_frames_to_holdset(row['frames'], ph_dict)
        for h in [h for l in holdset.values() for h in l]:
            if h not in grades_dict:
                grades_dict[h] = []
            grades_dict[h].append(row['difficulty_average'])
    except:
        continue
scaler = MinMaxScaler(feature_range=(35,100))
df_hold_diff = pd.DataFrame(np.array([[k,1/np.mean(v)] for k, v in grades_dict.items()]), columns=['hold_index','difficulty_level']).set_index('hold_index')
df_hold_diff['difficulty_level'] = np.round(scaler.fit_transform(df_hold_diff[['difficulty_level']])/100,2)
df_hold_diff.sort_values('difficulty_level')

,difficulty_level
hold_index,
4426.0,0.35
4295.0,0.35
4270.0,0.36
4405.0,0.37
4317.0,0.37
...,...
4263.0,0.97
1188.0,0.97
1186.0,0.99


In [204]:
endpoint = f"{API_BASE_URL}/api/v1/walls/wall-e77a688049fb"
response = requests.get(endpoint,)
content = json.loads(response.text)
holds = content['holds']
holds

[{'hold_index': 1133,
  'x': 13.666666666666666,
  'y': 0.3333333333333333,
  'pull_x': 0.0,
  'pull_y': -1.0,
  'useability': 0.5,
  'is_foot': 1},
 {'hold_index': 1134,
  'x': 13.333333333333336,
  'y': 0.6666666666666665,
  'pull_x': -0.5854905538443564,
  'pull_y': -0.8106792283998824,
  'useability': 0.5,
  'is_foot': 1},
 {'hold_index': 1135,
  'x': 13.0,
  'y': 0.3333333333333333,
  'pull_x': 0.0,
  'pull_y': -1.0,
  'useability': 0.5,
  'is_foot': 1},
 {'hold_index': 1136,
  'x': 12.666666666666664,
  'y': 0.6666666666666665,
  'pull_x': 0.3309722509530215,
  'pull_y': -0.9436404872084974,
  'useability': 0.5,
  'is_foot': 1},
 {'hold_index': 1137,
  'x': 12.333333333333334,
  'y': 0.3333333333333333,
  'pull_x': 0.0,
  'pull_y': -1.0,
  'useability': 0.5,
  'is_foot': 1},
 {'hold_index': 1138,
  'x': 12.0,
  'y': 0.6666666666666665,
  'pull_x': 0.6346632420772848,
  'pull_y': -0.7727888257188699,
  'useability': 0.5,
  'is_foot': 1},
 {'hold_index': 1139,
  'x': 11.66666666666

In [216]:
for hold in holds:
    try:
        hold['useability'] = float(df_hold_diff.loc[float(hold['hold_index']),'difficulty_level'])
    except:
        continue
upload_holds('wall-e77a688049fb',holds)

Uploading 641 holds to wall wall-e77a688049fb...
Endpoint: http://localhost:8000/api/v1/walls/wall-e77a688049fb/holds
✓ Successfully uploaded 641 holds!


{'id': 'wall-e77a688049fb'}

0.35